<a href="https://colab.research.google.com/github/Bhavya-1245/Image-Classifier-Detect---Cats-and-Dogs/blob/main/Object_Detection_Web_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gradio as gr
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from PIL import Image, ImageDraw, ImageFont

# --- 1. LOAD PRE-TRAINED OBJECT DETECTION MODEL ---
# We are NOT using our trained classifier. We are loading a
# powerful, pre-trained detection model from TensorFlow Hub.
# This model (Faster R-CNN) is more accurate but slower than MobileNetV2.
print("Loading pre-trained object detection model (This may take longer)...")
model_url = "https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1"
detector = hub.load(model_url)
print("Model loaded.")

# COCO dataset labels (what this model was trained on)
# We only care about 'cat' (ID 17) and 'dog' (ID 18)
COCO_LABELS = {
    17: "Cat",
    18: "Dog"
}

# --- 2. DEFINE THE DETECTION FUNCTION ---

def detect_objects(image_pil):
    """
    Takes a PIL image, runs detection, and returns the image
    with bounding boxes drawn on it.
    """
    if image_pil is None:
        return None

    # Convert PIL Image to NumPy array
    image_np = np.array(image_pil)

    # Convert to a tensor
    # Add a batch dimension: [height, width, 3] -> [1, height, width, 3]
    image_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]

    # The model expects uint8 tensor
    image_tensor = tf.image.convert_image_dtype(image_tensor, tf.uint8)

    # --- Run Detection ---
    results = detector(image_tensor)

    # The output is a dictionary. We take the items from the first (and only) image.
    result = {key: value.numpy()[0] for key, value in results.items()}

    # Get image dimensions
    im_width, im_height = image_pil.size

    # Prepare to draw on the image
    draw = ImageDraw.Draw(image_pil)

    # Try to load a font (optional, but looks good)
    try:
        font = ImageFont.load_default(size=16)
    except IOError:
        font = ImageFont.load_default()

    # Loop over all detections
    for i in range(len(result["detection_scores"])):
        score = result["detection_scores"][i]
        class_id = int(result["detection_classes"][i])
        box = result["detection_boxes"][i]

        # --- Filter for Cats and Dogs with > 50% confidence ---
        if score > 0.5 and class_id in COCO_LABELS:
            label = COCO_LABELS[class_id]

            # Get box coordinates in pixels
            # The model returns normalized coordinates (0.0 to 1.0)
            ymin, xmin, ymax, xmax = box
            left = xmin * im_width
            right = xmax * im_width
            top = ymin * im_height
            bottom = ymax * im_height

            # --- Draw the Box and Label ---
            color = 'red' if label == 'Dog' else 'lime'

            # Draw rectangle
            draw.rectangle([(left, top), (right, bottom)], outline=color, width=3)

            # Draw label background
            text = f"{label}: {int(score * 100)}%"
            text_bbox = draw.textbbox((left, top), text, font=font)
            draw.rectangle(text_bbox, fill=color)

            # Draw text
            draw.text((left, top), text, fill='black', font=font)

    return image_pil

# --- 3. CREATE THE GRADIO WEB APP ---

print("Starting Gradio app... Access it at http://127.0.0.1:7860")

# Create the Gradio interface
app = gr.Interface(
    fn=detect_objects,
    inputs=gr.Image(type="pil", label="Upload an Image"),
    outputs=gr.Image(type="pil", label="Detected Objects"),
    title="Project: Cat AND Dog Detector",
    description="This app uses a pre-trained SSD MobileNetV2 model from TensorFlow Hub to find cats and dogs in an image and draw boxes around them."
)

# Launch the app!
app.launch()

Loading pre-trained object detection model (This may take longer)...
Model loaded.
Starting Gradio app... Access it at http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://458c62f367973421d8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
